In [38]:
from playwright.async_api import async_playwright
import asyncio
import json
from pathlib import Path
from random import shuffle

In [39]:
def generate(nb):
    datas = "azertyuiopmlkjhgfdsqwxcvbn"
    datas += datas.upper() + "1234567890"
    datas = list(datas) * nb
    shuffle(datas)
    return "".join(datas[:nb])

In [40]:
async def main():
    async with async_playwright() as p:
        browser = await p.firefox.launch(slow_mo=3000)

        # Create context first
        context = await browser.new_context()

        # Create page inside the context
        page = await context.new_page()

        url = "https://witanime.cyou/episode/katainaka-no-ossan-kensei-ni-naru-%d8%a7%d9%84%d8%ad%d9%84%d9%82%d8%a9-7/"
        await page.goto(url, wait_until="domcontentloaded")

        # Click element via evaluate
        shortlink = await page.evaluate("""() => {
            const footer = document.querySelector("link[rel='shortlink']");
            p = footer.href.split('=').pop();
            return p;
        }""")


        # Click element via evaluate
        servers = await page.evaluate("""() => {
            const footer = document.querySelectorAll('.server-link');
            return footer;
        }""")

        i=0

        iframe = ""
        iframes = {}

        while i<len(servers):
            
            # Click element via evaluate
            key = await page.evaluate("""(index) => {
                const footer = document.querySelectorAll('.server-link');
                footer[index].click();
                return footer[index].textContent.trim();
            }""", i)

            i+=1
            
            # Wait for iframe or whatever needs to load
            await asyncio.sleep(1)  # Prefer asyncio.sleep in async code
    
            # Now get all pages in the same context (including your original page + new tabs if any)
            pages = context.pages
    
            # Close all tabs but the first one
            for p in pages[1:]:
                await p.close()
            while True:
                # Get iframe src on the first page
                iframe_src = await page.evaluate("""() => {
                    const iframe = document.querySelector('#iframe-container').querySelector('iframe');
                    return iframe.src;
                }""")
                if iframe != iframe_src:
                    iframe = iframe_src
                    break
                await asyncio.sleep(0.5)
            print(">", key)
            iframes[key]=iframe_src
            

        # Keep browser open for a while to observe
        
        await browser.close()
        return shortlink, iframes

# dd = await main()

In [41]:
parent = Path("/home/mohamed/Documents/datas/Docker")
fn = parent / "cours.json"

with fn.open("r") as fl:
    datas = json.load(fl)

In [42]:
datas[0].keys()

dict_keys(['_id', 'chapter', 'chapter_id', 'code', 'content', 'nb', 'title'])

In [43]:
cours = {}
chps = {}
for data in datas:
    code = generate(20)
    key = data["chapter_id"]
    cours[code] = {
        "title": data["title"],
        "chapter": key,
        "content": data["content"]
    }
    chps[key]={
        "name": data["chapter"],
        "level": "All"
    }

In [44]:
with Path("/home/mohamed/Documents/datas/Docker/s_cours.json").open("w") as fl:
    json.dump(cours, fl)
with Path("/home/mohamed/Documents/datas/Docker/s_chps.json").open("w") as fl:
    json.dump(chps, fl)